In [49]:
import pandas as pd

In [50]:
import glob
import re

In [51]:
#AdvancedCNN_linear_b128_c10_t05.parquet

In [63]:
# List all files that match the broader pattern
files = glob.glob('../../metrics/epoch_metrics/*b256*.parquet')

In [64]:
len(files)

241

In [65]:
# Filter out the files based on the specific pattern
filtered_files = [f for f in files if re.search(r'(naive|linear|sketch)', f)]

In [66]:
len(filtered_files)

181

In [67]:
for f in filtered_files:
    df = pd.read_parquet(f)
    df2 = df[df['epoch'] <= 1000]
    
    df2.to_parquet(f)